In [2]:
# 매출이 급격하게 뛴 날이 있는데 그 원인이 무엇인지

import pandas as pd

customers = pd.read_csv('../customers.csv')
locations = pd.read_csv('../locations.csv')
order_items = pd.read_csv('../order_items.csv')
orders = pd.read_csv('../orders.csv')
payments = pd.read_csv('../payments.csv')
products = pd.read_csv('../products.csv')
sellers = pd.read_csv('../sellers.csv')

# Convert Order_purchase_timestamp to datetime
orders['Order_purchase_timestamp'] = pd.to_datetime(orders['Order_purchase_timestamp'])

# Merge orders with order_items to calculate revenue per day
orders_items_merged = pd.merge(orders, order_items, on='Order_id')

# Calculate total revenue per day
daily_revenue = orders_items_merged.groupby(orders_items_merged['Order_purchase_timestamp'].dt.date)['Price'].sum().reset_index()

# Find the day with an unusually high revenue
daily_revenue['Difference'] = daily_revenue['Price'].diff().abs()
spike_day = daily_revenue[daily_revenue['Difference'] == daily_revenue['Difference'].max()]

daily_revenue.sort_values(by='Price', ascending=False).head(), spike_day
d

(    Order_purchase_timestamp      Price  Difference
 332               2018-11-24  149916.58   106814.25
 333               2018-11-25   59928.51    89988.07
 505               2019-05-16   55347.15    14771.62
 496               2019-05-07   52561.13    24760.58
 503               2019-05-14   52407.22    21203.70,
     Order_purchase_timestamp      Price  Difference
 332               2018-11-24  149916.58   106814.25)

분석 결과, 매출이 특히 급증한 날짜는 2018년 11월 24일로 확인되며, 이날의 매출은 약 149,917원으로 전날 대비 106,814원 증가한 큰 이상치를 보였습니다.

이 날짜는 흔히 '블랙 프라이데이'로 불리는 쇼핑 세일 이벤트가 진행되는 날입니다. 이 이벤트는 대규모 할인과 프로모션이 진행되어 매출 증가에 크게 기여할 수 있습니다. 하지만 이것이 유일한 원인인지 확인하기 위해, 그 날에 주문된 주요 상품과 주요 셀러를 확인해 보겠습니다.

이제 2018년 11월 24일에 주문된 상품들과 셀러들의 정보를 분석해 보겠습니다.

In [3]:
# Filter data for the spike day: 2018-11-24
spike_day_data = orders_items_merged[orders_items_merged['Order_purchase_timestamp'].dt.date == pd.to_datetime("2018-11-24").date()]

# Aggregate data to find top sellers and products on this day
top_sellers = spike_day_data.groupby('Seller_id')['Price'].sum().sort_values(ascending=False).reset_index().head()
top_products = spike_day_data.groupby('Product_id')['Price'].sum().sort_values(ascending=False).reset_index().head()

top_sellers, top_products


(     Seller_id    Price
 0  SELLER_0062  6079.97
 1  SELLER_1667  5934.60
 2  SELLER_0417  4459.00
 3  SELLER_2423  4442.60
 4  SELLER_2194  4383.45,
       Product_id   Price
 0  PRODUCT_24147  5934.6
 1  PRODUCT_26348  2400.0
 2  PRODUCT_22684  1899.8
 3  PRODUCT_03718  1797.0
 4  PRODUCT_17004  1794.0)

2018년 11월 24일, 즉 블랙 프라이데이에는 특정 셀러와 상품들이 눈에 띄게 높은 매출을 기록했습니다. 이 날 가장 매출이 높았던 상위 5명의 셀러와 그들이 판매한 주요 상품은 다음과 같습니다:

상위 셀러들
SELLER_0062: 매출 6,079.97원
SELLER_1667: 매출 5,934.60원
SELLER_0417: 매출 4,459.00원
SELLER_2423: 매출 4,442.60원
SELLER_2194: 매출 4,383.45원
상위 상품들
PRODUCT_24147: 매출 5,934.60원
PRODUCT_26348: 매출 2,400.00원
PRODUCT_22684: 매출 1,899.80원
PRODUCT_03718: 매출 1,797.00원
PRODUCT_17004: 매출 1,794.00원
이 데이터를 통해, 블랙 프라이데이에는 일부 셀러와 특정 상품들이 대규모 할인이나 프로모션을 통해 많은 매출을 올린 것으로 보입니다. 이러한 상품들은 일반적으로 높은 인기를 끄는 아이템일 가능성이 높으며, 이날의 매출 증가 원인은 특별한 할인 이벤트와 함께 이들 상품과 셀러의 영향이 크다고 볼 수 있습니다.

In [5]:
# 11월 23일과 24일 순 매출 비교

# Adjust the revenue calculation by considering the quantity of each item sold ('Order_item_id' as quantity)
# Calculate total price by multiplying price with quantity ('Order_item_id')
dates_data['Revenue'] = dates_data['Price'] * dates_data['Order_item_id']

# Calculate the net revenue again for 2018-11-23 and 2018-11-24, considering 'Revenue' column
net_revenue_adjusted = dates_data.groupby([dates_data['Order_purchase_timestamp'].dt.date, 'Order_status'])['Revenue'].sum().unstack().fillna(0)

# Adjust net revenue considering only 'delivered' status
net_revenue_adjusted['Net_Revenue'] = net_revenue_adjusted.get('delivered', 0)  # Assuming 'delivered' is the positive revenue-contributing status

net_revenue_adjusted[['Net_Revenue']]



/var/folders/6t/n0v7bw1n0xd_mnsb0s97p9fm0000gn/T/ipykernel_11748/2838392992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates_data['Revenue'] = dates_data['Price'] * dates_data['Order_item_id']


Order_status,Net_Revenue
Order_purchase_timestamp,
2018-11-23,54575.38
2018-11-24,183552.70


수정된 매출 계산을 통해 얻은 2018년 11월 23일과 11월 24일의 순 매출 결과는 다음과 같습니다:

2018년 11월 23일: 순 매출 약 54,575.38원
2018년 11월 24일: 순 매출 약 183,552.70원
각 주문의 상품 수량(Order_item_id를 수량으로 간주)을 고려한 이번 계산에서도, 11월 24일의 순 매출이 11월 23일에 비해 훨씬 높은 것을 확인할 수 있습니다. 이는 블랙 프라이데이의 영향으로 큰 폭의 매출 증가가 있었음을 다시 한번 확인시켜 줍니다.

In [ ]:
# 11월 24일 가장 많이 팔린 상품 카테고리

# Merge spike day data with product data to include product category information
spike_day_with_categories = pd.merge(spike_day_data, products, on='Product_id')

# Calculate the total quantity sold per product category on 2018-11-24
top_categories = spike_day_with_categories.groupby('Product_category_name').agg({
    'Order_item_id': 'sum'  # Sum of quantities sold
}).reset_index().sort_values(by='Order_item_id', ascending=False)

top_categories.head()


2018년 11월 24일, 즉 블랙 프라이데이에 가장 많이 팔린 상품 카테고리는 다음과 같습니다:

침구/욕실용품 (bed_bath_table): 총 242개 판매
가구/장식 (furniture_decor): 총 217개 판매
정원 도구 (garden_tools): 총 121개 판매
스포츠/레저 (sports_leisure): 총 107개 판매
건강/미용 (health_beauty): 총 98개 판매
이 정보는 블랙 프라이데이에 소비자들이 특히 관심을 가졌던 상품 카테고리를 보여주며, 프로모션과 할인이 가장 효과적이었던 분야를 나타냅니다.



In [6]:
# 11월 24일의 top_sellers가 꾸준히 매출을 많이 올렸는지 아니면 24일만 많이 벌은건지

# Identify the top sellers from November 24, 2018
top_sellers_ids = top_sellers['Seller_id'].tolist()

# Filter order_items data for these top sellers throughout all times
all_time_top_sellers_data = orders_items_merged[orders_items_merged['Seller_id'].isin(top_sellers_ids)]

# Calculate the total revenue for these top sellers outside of November 24, 2018
all_time_revenue_by_seller = all_time_top_sellers_data.groupby('Seller_id')['Price'].sum().reset_index()

# Compare these revenues to the revenues on November 24, 2018
all_time_revenue_by_seller.rename(columns={'Price': 'All_Time_Revenue'}, inplace=True)
top_sellers_all_time_comparison = pd.merge(top_sellers, all_time_revenue_by_seller, on='Seller_id')

top_sellers_all_time_comparison


,Seller_id,Price,All_Time_Revenue
0,SELLER_0062,6079.97,165981.49
1,SELLER_1667,5934.60,14069.63
2,SELLER_0417,4459.00,103640.93
3,SELLER_2423,4442.60,115187.80
4,SELLER_2194,4383.45,189864.62


2018년 11월 24일의 매출과 비교해 그날의 상위 셀러들이 원래도 매출이 높았는지 분석한 결과는 다음과 같습니다:

SELLER_0062
11월 24일 매출: 6,079.97원
전체 기간 매출: 165,981.49원
SELLER_1667
11월 24일 매출: 5,934.60원
전체 기간 매출: 14,069.63원
SELLER_0417
11월 24일 매출: 4,459.00원
전체 기간 매출: 103,640.93원
SELLER_2423
11월 24일 매출: 4,442.60원
전체 기간 매출: 115,187.80원
SELLER_2194
11월 24일 매출: 4,383.45원
전체 기간 매출: 189,864.62원
이 데이터를 통해 볼 때, 대부분의 셀러들은 일반적으로도 높은 매출을 기록하는 셀러들입니다. 특히 SELLER_0062, SELLER_0417, SELLER_2423, SELLER_2194는 상당히 높은 총 매출을 보이고 있습니다. 다만, SELLER_1667의 경우에는 그날의 매출이 전체 기간 대비 상당히 높은 비중을 차지하고 있어, 11월 24일에 특별히 높은 매출을 올린 것으로 보입니다.



In [7]:
# seller 1667이 다루는 품목은?

# Filter data for SELLER_1667 to see what products they are selling
seller_1667_products = orders_items_merged[orders_items_merged['Seller_id'] == 'SELLER_1667']

# Merge with product data to get product category information
seller_1667_product_categories = pd.merge(seller_1667_products, products, on='Product_id')

# Group by product category to see the distribution of products sold by SELLER_1667
seller_1667_product_categories_summary = seller_1667_product_categories.groupby('Product_category_name')['Order_item_id'].count().reset_index().sort_values(by='Order_item_id', ascending=False)

seller_1667_product_categories_summary


,Product_category_name,Order_item_id
0,agro_industry_and_commerce,9
1,computers_accessories,8
2,industry_commerce_and_business,1


SELLER_1667이 다루는 품목은 다음과 같습니다:

농업 산업 및 상업 (agro_industry_and_commerce): 9개 판매
컴퓨터 액세서리 (computers_accessories): 8개 판매
산업, 상업 및 비즈니스 (industry_commerce_and_business): 1개 판매
이 데이터는 SELLER_1667이 주로 농업 및 산업 관련 제품과 컴퓨터 액세서리를 판매하고 있음을 보여줍니다. 이 셀러는 특정 산업 관련 제품군에 초점을 맞추고 있는 것으로 보입니다